# Problem 1

### Part a

In [6]:
from scipy.stats import poisson

# Prior probabilities
prior_probs = [1/3, 1/3, 1/3]

# Occurrence rates given by the experts
rates = [1.0e-4, 5.0e-3, 0.05]

# Number of earthquakes observed
k = 3

# Total time period
T = 2000

# Calculate likelihoods
likelihoods = [poisson.pmf(k, rate*T) for rate in rates]

# Calculate unnormalized posterior probabilities
post_probs_unnorm = [prior*likelihood for prior, likelihood in zip(prior_probs, likelihoods)]

# Normalize the posterior probabilities
post_probs = [prob/sum(post_probs_unnorm) for prob in post_probs_unnorm]

# Compute the weighted average of the occurrence rates
weighted_avg_rate = sum(post_prob*rate for post_prob, rate in zip(post_probs, rates))

print(f"Posterior probabilities: {post_probs}")
print(f"Weighted average occurrence rate: {weighted_avg_rate}")

Posterior probabilities: [0.1260803521359797, 0.8739196478640202, 7.160908626951125e-37]
Weighted average occurrence rate: 0.004382206274533698


### Part b

In [7]:
from scipy.stats import poisson

# Weighted average occurrence rate from part a
rate = 0.004382206274533698

# Total time period
T = 60

# Calculate the probability of zero earthquakes
prob_zero = poisson.pmf(0, rate*T)

# Calculate the probability of at least one earthquake
prob_at_least_one = 1 - prob_zero

print(f"Probability of at least one earthquake: {prob_at_least_one}")

Probability of at least one earthquake: 0.23120611589189344


# Problem 2

In [8]:
# Data
years_of_operation = [35, 40, 30]
number_of_fires = [3, 2, 0]

# Total number of fires
total_fires = sum(number_of_fires)

# Total years of operation
total_years = sum(years_of_operation)

# Calculate the MLE for the fire ignition frequency
lambda_mle = total_fires / total_years

print(f"Point estimate of λ [1/year]: {lambda_mle}")

Point estimate of λ [1/year]: 0.047619047619047616


# Problem 3

In [9]:
from scipy.stats import binom

# Prior probabilities and failure probabilities per demand
prior_probs = [0.5, 0.5]
failure_probs = [[0.01, 0.03, 0.05], [0.02, 0.05, 0.10]]
prob_masses = [[0.25, 0.50, 0.25], [0.60, 0.20, 0.20]]

# Test data
n = 100
k = 2

# Calculate likelihoods
likelihoods = [[binom.pmf(k, n, p) for p in failure_probs[i]] for i in range(2)]

# Calculate unnormalized posterior probabilities for each failure probability
post_probs_unnorm = [[prob_mass * likelihood for prob_mass, likelihood in zip(prob_masses[i], likelihoods[i])] for i in range(2)]

# Normalize the posterior probabilities for each failure probability
post_probs = [[prob/sum(post_probs_unnorm[i]) for prob in post_probs_unnorm[i]] for i in range(2)]

# Calculate the total unnormalized posterior probabilities for each distribution
total_post_probs_unnorm = [prior_prob * sum(post_probs_unnorm[i]) for i, prior_prob in enumerate(prior_probs)]

# Normalize the posterior probabilities for each distribution
total_post_probs = [prob/sum(total_post_probs_unnorm) for prob in total_post_probs_unnorm]

print(f"Posterior probabilities for each failure probability: {post_probs}")
print(f"Posterior probabilities for each distribution: {total_post_probs}")

Posterior probabilities for each failure probability: [[0.2580640321113161, 0.6286094003312103, 0.1133265675574736], [0.90830488814569, 0.08989764490936077, 0.0017974669449492796]]
Posterior probabilities for each distribution: [0.4978854292393337, 0.5021145707606663]


# Problem 4

In [10]:
import numpy as np
from scipy.stats import uniform, poisson, norm

# Field data
years_of_operation = np.array([35., 40., 30.])
number_of_fires = np.array([3., 2., 0.])

# Prior distributions for mu and sigma
prior_mu = uniform(loc=-7.0, scale=3.0)  # Uniform between -7 and -4
prior_sigma = uniform(loc=0.5, scale=1.0)  # Uniform between 0.5 and 1.5

# Log likelihood function
def log_likelihood(mu, sigma, years_of_operation, number_of_fires):
    lambda_ = np.exp(norm.rvs(loc=mu, scale=sigma, size=len(years_of_operation)))
    return poisson.logpmf(number_of_fires, mu=lambda_ * years_of_operation).sum()

# Metropolis-Hastings algorithm
def metropolis_hastings(prior_mu, prior_sigma, log_likelihood, iterations, initial_values, years_of_operation, number_of_fires):
    mu_current, sigma_current = initial_values
    accepts = 0

    for i in range(iterations):
        mu_proposal = norm.rvs(loc=mu_current, scale=0.5)
        sigma_proposal = norm.rvs(loc=sigma_current, scale=0.5)

        # Check if proposed sigma is within the support of the prior
        if sigma_proposal < 0.5 or sigma_proposal > 1.5:
            continue

        likelihood_current = log_likelihood(mu_current, sigma_current, years_of_operation, number_of_fires)
        likelihood_proposal = log_likelihood(mu_proposal, sigma_proposal, years_of_operation, number_of_fires)

        prior_current = prior_mu.logpdf(mu_current) + prior_sigma.logpdf(sigma_current)
        prior_proposal = prior_mu.logpdf(mu_proposal) + prior_sigma.logpdf(sigma_proposal)

        p_current = likelihood_current + prior_current
        p_proposal = likelihood_proposal + prior_proposal

        p_accept = np.exp(p_proposal - p_current)

        accept = p_accept > np.random.rand()
        if accept:
            mu_current = mu_proposal
            sigma_current = sigma_proposal
            accepts += 1

    return mu_current, sigma_current, accepts / iterations

# Run the Metropolis-Hastings algorithm
mu, sigma, acceptance_rate = metropolis_hastings(prior_mu, prior_sigma, log_likelihood, 5000, (-5.5, 1.0), years_of_operation, number_of_fires)

print(f"mu: {mu}")
print(f"sigma: {sigma}")
print(f"Acceptance rate: {acceptance_rate}")

mu: -4.092744766213293
sigma: 0.5780060137126292
Acceptance rate: 0.2774
